Positional Encoding

In [1]:
import numpy as np
import math
def positional_encoding(max_len, d_model):
    # max_len: độ dài tối đa của 1 chuỗi
    # d_model: kích thước embedding
    pe = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            angle = pos / (10000 ** (i / d_model))
            # Vị trí chẵn
            pe[pos, i] = math.sin(angle)
            # Vị trí lẻ
            pe[pos, i+1] = math.cos(angle)
    return pe



In [2]:
max_len = 5
d_model = 4
pe = np.zeros((5, 4))
print(f"ma trận ban đầu:\n{pe}")
print(f"------------------")
for pos in range(max_len):
    for i in range(0, d_model, 2):
        angle = pos / (10000 ** (i / d_model))
        pe[pos][i] = math.sin(angle)
        pe[pos][i+1] = math.cos(angle)
print(f"ma trận positional encoding:\n{pe}")
print(f"--------------------------")
for i in range(max_len):
    print(f"vector PE cho vị trí thứ: {i} là:\n{pe[i]}")
    print(f"------------------------------")


ma trận ban đầu:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
------------------
ma trận positional encoding:
[[ 0.          1.          0.          1.        ]
 [ 0.84147098  0.54030231  0.00999983  0.99995   ]
 [ 0.90929743 -0.41614684  0.01999867  0.99980001]
 [ 0.14112001 -0.9899925   0.0299955   0.99955003]
 [-0.7568025  -0.65364362  0.03998933  0.99920011]]
--------------------------
vector PE cho vị trí thứ: 0 là:
[0. 1. 0. 1.]
------------------------------
vector PE cho vị trí thứ: 1 là:
[0.84147098 0.54030231 0.00999983 0.99995   ]
------------------------------
vector PE cho vị trí thứ: 2 là:
[ 0.90929743 -0.41614684  0.01999867  0.99980001]
------------------------------
vector PE cho vị trí thứ: 3 là:
[ 0.14112001 -0.9899925   0.0299955   0.99955003]
------------------------------
vector PE cho vị trí thứ: 4 là:
[-0.7568025  -0.65364362  0.03998933  0.99920011]
------------------------------


Sau khi tính PE, ta cộng vào embedding ban đầu: X' = X + PE

In [3]:
# Giả sử vector X ban đầu có giá trị:
x = np.random.randn(max_len, d_model)
print(f"vector embedding của input:\n{x}")
x = x + pe
print(f"---------------------------------")
print(f"vector x khi thêm positional encoding:\n{x}")

vector embedding của input:
[[-0.45413692  0.29019407 -1.34147064 -0.23869127]
 [-0.57477958 -1.12945031 -0.51195096  0.55844638]
 [ 0.40192628 -0.10812314 -0.32817709  0.60401348]
 [ 1.30145968  1.36210844  1.55611376  0.21600585]
 [ 1.71018055  0.94075442  0.88960112 -1.40731508]]
---------------------------------
vector x khi thêm positional encoding:
[[-0.45413692  1.29019407 -1.34147064  0.76130873]
 [ 0.2666914  -0.58914801 -0.50195113  1.55839638]
 [ 1.31122371 -0.52426998 -0.30817842  1.60381348]
 [ 1.44257969  0.37211594  1.58610926  1.21555589]
 [ 0.95337806  0.2871108   0.92959046 -0.40811497]]


Multihead Attention

Residual and Normalization Layer

In [4]:
import torch
import torch.nn as nn
# Layer normalization
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        # gamma và beta
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased= False)

        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        return self.gamma * x_hat + self.beta

In [5]:
d_model = 5
gamma1 = nn.Parameter(torch.zeros(d_model))
gamma2 = torch.zeros(d_model)
print(f"gamma1:\n{gamma1}")
print(f"gamma2:\n{gamma2}")
print(f"---------------------------")
matrix = torch.zeros(3,4)
print(matrix)

gamma1:
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
gamma2:
tensor([0., 0., 0., 0., 0.])
---------------------------
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [6]:
# Tạo input gồm 2 câu batch_size = 2
batch_size = 2
seq_len = 3
d_model = 5
x = torch.randn(batch_size,seq_len, d_model)
print(x)
mean = x.mean(dim=-1, keepdim=True)
print(f"Trung bình của từng hàng: \n{mean}")
var = x.var(dim = -1, keepdim=True, unbiased=False)
print(f"Phương sai của từng hàng:\n{var}")

tensor([[[ 0.1232, -0.3697, -0.1003,  0.4852, -2.1945],
         [ 1.6909, -1.5506, -0.3908, -1.5257, -1.9677],
         [ 1.0208,  1.4066, -1.7473, -1.0091, -1.5697]],

        [[ 2.3933, -1.4482, -1.0889, -0.4912, -1.3202],
         [ 0.7352, -0.1250, -0.0594, -2.1032,  1.2380],
         [-0.3301,  0.2461,  0.6317, -1.0883, -0.2924]]])
Trung bình của từng hàng: 
tensor([[[-0.4112],
         [-0.7488],
         [-0.3797]],

        [[-0.3910],
         [-0.0629],
         [-0.1666]]])
Phương sai của từng hàng:
tensor([[[0.8735],
         [1.7625],
         [1.7669]],

        [[2.0461],
         [1.2992],
         [0.3399]]])


In [7]:
gamma = torch.randn(d_model)
print(f"gamma:\n{gamma}")
x_hat = (x - mean) / torch.sqrt(var + 1e-6)
print(f"Chuẩn hóa các giá trị của input:\n{x_hat}")
ans = gamma * x_hat + gamma
print(f"Đầu ra layernorm của input:\n{ans}")

gamma:
tensor([-1.5260,  1.7643, -2.1167,  0.8053, -0.8014])
Chuẩn hóa các giá trị của input:
tensor([[[ 0.5718,  0.0444,  0.3326,  0.9591, -1.9080],
         [ 1.8377, -0.6040,  0.2697, -0.5852, -0.9181],
         [ 1.0536,  1.3439, -1.0288, -0.4734, -0.8952]],

        [[ 1.9465, -0.7390, -0.4879, -0.0700, -0.6496],
         [ 0.7002, -0.0545,  0.0030, -1.7900,  1.1413],
         [-0.2805,  0.7079,  1.3692, -1.5808, -0.2158]]])
Đầu ra layernorm của input:
tensor([[[-2.3986,  1.8426, -2.8208,  1.5777,  0.7277],
         [-4.3302,  0.6987, -2.6875,  0.3340, -0.0656],
         [-3.1337,  4.1353,  0.0609,  0.4241, -0.0840]],

        [[-4.4963,  0.4604, -1.0839,  0.7490, -0.2808],
         [-2.5944,  1.6682, -2.1231, -0.6362, -1.7160],
         [-1.0979,  3.0132, -5.0149, -0.4677, -0.6284]]])


In [8]:
# Sublayer: Feed forward network: 2 hidden layer
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        output = self.linear2(x)
        return output

In [9]:
d_model = 4
d_ff = 6
linear1 = nn.Linear(d_model, d_ff)
print(linear1)

Linear(in_features=4, out_features=6, bias=True)


In [10]:
# Residual connection Layernorm
class SublayerConnection(nn.Module):
    # y = layernorm(x + sublayer(x))
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, sublayer):
        output = self.norm(x + self.dropout(sublayer(x)))
        return output

In [11]:
dropout = nn.Dropout(0.3)
print(dropout)

Dropout(p=0.3, inplace=False)


In [12]:
# y = layernorm(x + sublayer(x))
# Giả sử input x là 1 câu có 4 từ, mỗi từ được mã hóa thành vector 6 chiều
# ma trận embedding
x = torch.randn(1, 4, 6)
print(f"input:\n{x}")
sublayer = FeedForward(d_model=6, d_ff=10)
sublayer_connection = SublayerConnection(6)
y = sublayer_connection(x, sublayer)
print(f"output của x khi đi qua residual + layer normalization:\n{y}")

input:
tensor([[[-0.6801,  0.2333,  0.4238, -0.4792,  0.2576, -0.5232],
         [ 0.1243, -0.6267,  0.5940,  0.6381, -0.4827, -0.2550],
         [ 0.5542,  0.0287, -0.9203, -0.8005,  0.1908,  0.8928],
         [-0.3720, -2.0090,  0.0971,  1.3827, -2.2678, -0.2903]]])
output của x khi đi qua residual + layer normalization:
tensor([[[-0.9000, -0.3499,  1.7484, -0.3912,  0.9087, -1.0159],
         [ 0.2296, -1.6538,  1.3579,  0.9629, -0.3083, -0.5883],
         [ 1.0469, -0.7939, -0.9680, -1.2083,  0.8452,  1.0781],
         [ 0.1839, -1.3701,  1.0333,  1.3823, -1.0359, -0.1936]]],
       grad_fn=<AddBackward0>)


Cài đặt MHA

In [15]:
def softmax(x):
    # softmax theo hàng
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x/np.sum(e_x, axis=-1, keepdims=True)
def scaled_dot_product_attention(Q, K, V):
    # Q: (n, d_k) - n query vectors
    # K: (m, d_k) - m key vectors
    # V: (m, d_v) - m value vectors
    # Lấy ra d_k cho bước tính căn bậc 2
    d_k = Q.shape[1]
    # Tính score
    scores = np.matmul(Q, K.T) / np.sqrt(d_k)
    # Tính trọng số attention
    weights = softmax(scores)
    # Tổ hợp tuyến tính với V
    output = np.matmul(weights, V)
    return output, weights

In [ ]:
class MultiHeadAttention:
    def __init__(self, d_model, num_heads):
        # d_model phải chia hết cho num_heads
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        # Khởi tạo ma trận trọng số
        self.W_Q = np.random.randn(d_model, d_model)
        self.W_K = np.random.randn(d_model, d_model)
        self.W_V = np.random.randn(d_model, d_model)
        # Ma trận trọng số được sử dụng cuối cùng
        self.W_O = np.random.randn(d_model, d_model)
    def split_heads(self, x):
        # x: (seq_len, d_model)
        # return: (num_heads, seq_len, d_k)
        seq_len = x.shape[0]
        # Tách X thành 1 mảng gồm seq_len ma trận, mỗi ma trận có num_heads hàng, d_k cột
        x = x.reshape(seq_len, self.num_heads, self.d_k)
    def combine_heads(self, x):
        # x: (num_heads, seq_len, d_k)
        # return: (seq_len, d_model)
        num_heads, seq_len, d_k = x.shape
        x = x.transpose(1, 0, 2).reshape(seq_len, self.d_model)
    def forward(self, Q, K, V):
        # linear projection
        Q_proj = Q @ self.W_Q
        K_proj = K @ self.W_K
        V_proj = V @ self.W_V
        # Chia thành nhiều head
        Q_heads = self.split_heads(Q_proj)
        K_heads = self.split_heads(K_proj)
        V_heads = self.split_heads(V_proj)
        # Attention trên từng head
        head_outputs = []
        for i in range(self.num_heads):
            out = scaled_dot_product_attention(Q_heads[i], K_heads[i], V_heads[i])
            


Pipeline của Encoder trong transformer

In [ ]:
import numpy as np
class EncoderLayer:
    def __init__(self, d_model, d_ff, num_heads, dropout=0.1):
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        # Residual + layer Normalize (2 lần: sau MHA và sau FFN)
        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)

    def forward(self, x, mask=None):
        # self attention + residual và layer norm
        attn_ouput = 
        x = self.norm(x + attn_ouput)

        # ffn + residual và layer norm
        ff_output = self.feed_forward.forward(x)
        x = self.norm2(x + ff_output)
        return x